In [123]:
import cdapython as cda
import pandas as pd
#from cdapython import Q, columns, unique_terms, query

# Identify TCGA melanoma research subjects
# Identify which ones have DNA bams
# Identify hwich ones have RNA bams
# Starting at 9 pm, other sheet took 30 minutes to create

In [8]:
cda.columns()

['id',
 'identifier',
 'identifier.system',
 'identifier.value',
 'species',
 'sex',
 'race',
 'ethnicity',
 'days_to_birth',
 'subject_associated_project',
 'vital_status',
 'age_at_death',
 'cause_of_death',
 'File',
 'File.id',
 'File.identifier',
 'File.identifier.system',
 'File.identifier.value',
 'File.label',
 'File.data_category',
 'File.data_type',
 'File.file_format',
 'File.associated_project',
 'File.drs_uri',
 'File.byte_size',
 'File.checksum',
 'File.data_modality',
 'File.imaging_modality',
 'File.dbgap_accession_number',
 'ResearchSubject',
 'ResearchSubject.id',
 'ResearchSubject.identifier',
 'ResearchSubject.identifier.system',
 'ResearchSubject.identifier.value',
 'ResearchSubject.member_of_research_project',
 'ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubjec

In [182]:
cda.unique_terms('ResearchSubject.File.data_category')

['Biospecimen',
 'Clinical',
 'Combined Nucleotide Variation',
 'Copy Number Variation',
 'DNA Methylation',
 'Peptide Spectral Matches',
 'Processed Mass Spectra',
 'Protein Assembly',
 'Proteome Profiling',
 'Quality Metrics',
 'Raw Mass Spectra',
 'Sequencing Reads',
 'Simple Nucleotide Variation',
 'Somatic Structural Variation',
 'Structural Variation',
 'Transcriptome Profiling']

ResearchSubject.Specimen.File.label is some random strings with the suffix "dcm"

In [39]:
# how can we specify tcga melanoma? # Research Subject?
q_skcm = cda.query('ResearchSubject.member_of_research_project = "TCGA-SKCM"')
q_skcm = cda.Q('ResearchSubject.member_of_research_project = "TCGA-SKCM"')

# how can we specify a bam? # File format?
q_bam = cda.query('File.file_format = "BAM"')
q_bam = cda.Q('File.file_format = "BAM"')

# how can we specify WXS? ResearchSubject.Specimen? There does not seem to be an option to view technology type
# GDC has the category "Experimental Strategy" that lists sequencing types

# how can we specify tumor? ResearchSubject.Specimen
q_sequencing = cda.Q('ResearchSubject.File.data_category = "Sequencing Reads"')

q_sample = cda.query('ResearchSubject.Specimen.specimen_type = "sample"')
q_sample = cda.Q('ResearchSubject.Specimen.specimen_type = "sample"')

q_tumor = cda.query('ResearchSubject.Specimen.source_material_type = "Tumor"') # here, unique terms also have "primary tumor", "recurrent tumor", and "DNA"
q_tumor = cda.Q('ResearchSubject.Specimen.source_material_type = "Tumor"')

q = q_skcm.And(q_bam.And(q_sample.And(q_tumor)))
r = q.run()


Getting results from database



ERROR:root:
                Http Status: 500
                Error Message: Column name File is ambiguous at [1:124]
                


Total execution time: 2904 ms


In [44]:
query = 'ResearchSubject.member_of_research_project = "TCGA-SKCM" AND File.file_format = "BAM"'
q = cda.Q(query)
response = q.run()

Getting results from database

Total execution time: 2951 ms


In [45]:
response


        QueryID: 2c28508c-6c62-4e7b-a23b-1b8ba86b814c
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject WHERE (_ResearchSubject.member_of_research_project = 'TCGA-SKCM" AND File.file_format = "BAM')
        Offset: 0
        Count: 0
        Total Row Count: 0
        More pages: False
        

It seems that AND is not working. I wonder if it is because ResearchSubject is querying participants while File is querying files...

In [47]:
query = 'ResearchSubject.member_of_research_project = "TCGA-SKCM"'
q = cda.Q(query)
response_skcm = q.run()

query = 'File.file_format = "BAM"'
q = cda.Q(query)
response_bam = q.run()

Getting results from database

Total execution time: 11844 ms
Getting results from database

Total execution time: 19289 ms


In [48]:
response_skcm


        QueryID: 0d5a29ad-5df6-4ee0-9281-090b7867b7b1
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject WHERE (_ResearchSubject.member_of_research_project = 'TCGA-SKCM')
        Offset: 0
        Count: 100
        Total Row Count: 470
        More pages: True
        

In [49]:
response_bam


        QueryID: 4b75e01f-502a-42cf-b6a2-db81b0ada9d1
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(File) AS _File WHERE (_File.file_format = 'BAM')
        Offset: 0
        Count: 100
        Total Row Count: 87853
        More pages: True
        

# Working from a single query
Since it seems that we cannot combine queries, we are instead going to query against TCGA SKCM and see how far we can get with that initial query and response.

- Can we identify patients with melanoma?
- Can we identify patients with bams?
- Can we identify patients with WXS bams?
- Can we identify patients with RNA-seq bams?

In [135]:
query = 'ResearchSubject.member_of_research_project = "TCGA-SKCM"'
q = cda.Q(query)
r = q.run(limit=1000)

Getting results from database

Total execution time: 34979 ms


In [136]:
r


        QueryID: 45aeac13-20df-4f73-873e-bd9be7c16ede
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject WHERE (_ResearchSubject.member_of_research_project = 'TCGA-SKCM')
        Offset: 0
        Count: 470
        Total Row Count: 470
        More pages: False
        

As expected, we return 470 participants or "research subjects"

In [137]:
r[0].keys()

dict_keys(['id', 'identifier', 'species', 'sex', 'race', 'ethnicity', 'days_to_birth', 'subject_associated_project', 'vital_status', 'age_at_death', 'cause_of_death', 'File', 'ResearchSubject'])

In [138]:
r[0]['ResearchSubject'][0].keys()

dict_keys(['id', 'identifier', 'member_of_research_project', 'primary_diagnosis_condition', 'primary_diagnosis_site', 'Diagnosis', 'File', 'Specimen'])

In [139]:
len(r[0]['File'])
# 54 files associated with this individual

54

In [140]:
r[0]['File'][0]
# These is not a way to map back up to individual

{'id': 'c078143e-7eca-43a2-a639-730475f6ad49',
 'identifier': [{'system': 'GDC',
   'value': 'c078143e-7eca-43a2-a639-730475f6ad49'}],
 'label': 'c078143e-7eca-43a2-a639-730475f6ad49.vep.vcf.gz',
 'data_category': 'Simple Nucleotide Variation',
 'data_type': 'Annotated Somatic Mutation',
 'file_format': 'VCF',
 'associated_project': 'TCGA-SKCM',
 'drs_uri': 'drs://dg.4DFC:c078143e-7eca-43a2-a639-730475f6ad49',
 'byte_size': '1068604',
 'checksum': '2ffadb975ad20842936a51527c02f62e',
 'data_modality': 'Genomic',
 'imaging_modality': None,
 'dbgap_accession_number': None}

In [141]:
bams = {}
for research_subject in r[0:1]:
    research_subject_id = research_subject['id']
    files = research_subject['File']
    bams = [file for file in files if file['file_format'] == 'BAM'] 

In [142]:
bams[0]
# There is no way for us to identify more information about this file; e.g. sequencing type or sample type

{'id': '3f474ec5-1b6e-45e5-9fc4-fdb54e9a133b',
 'identifier': [{'system': 'GDC',
   'value': '3f474ec5-1b6e-45e5-9fc4-fdb54e9a133b'}],
 'label': 'TCGA-ER-A3EV-06A-11R-A20E-13_mirna_gdc_realn.bam',
 'data_category': 'Sequencing Reads',
 'data_type': 'Aligned Reads',
 'file_format': 'BAM',
 'associated_project': 'TCGA-SKCM',
 'drs_uri': 'drs://dg.4DFC:3f474ec5-1b6e-45e5-9fc4-fdb54e9a133b',
 'byte_size': '142366534',
 'checksum': '86bffb2922b60115bd0b1bf8e154a899',
 'data_modality': 'Genomic',
 'imaging_modality': None,
 'dbgap_accession_number': None}

In [143]:
len(r[0]['ResearchSubject'][0]['Specimen'])

22

In [144]:
# There does not seem to be a way to figure out more information about specimen. I'll list all source_material_types

In [145]:
r[0]['ResearchSubject'][0]['Specimen'][1]

{'id': 'a4a0d4ca-7909-568f-b880-3ed1d1401060',
 'identifier': [{'system': 'GDC',
   'value': 'a4a0d4ca-7909-568f-b880-3ed1d1401060'}],
 'associated_project': 'TCGA-SKCM',
 'age_at_collection': '-20354',
 'primary_disease_type': 'Nevi and Melanomas',
 'anatomical_site': None,
 'source_material_type': 'Primary Tumor',
 'specimen_type': 'portion',
 'derived_from_specimen': '4135df0c-983a-4565-b3e0-8056aa4be2bb',
 'derived_from_subject': 'TCGA-ER-A3EV',
 'File': [{'id': '10038031-4300-4c86-b51d-fa227585680f',
   'identifier': [{'system': 'GDC',
     'value': '10038031-4300-4c86-b51d-fa227585680f'}],
   'label': 'TCGA-ER-A3EV-01Z-00-DX1.295FE0BC-4BCC-4950-A671-01A49FE10185.svs',
   'data_category': 'Biospecimen',
   'data_type': 'Slide Image',
   'file_format': 'SVS',
   'associated_project': 'TCGA-SKCM',
   'drs_uri': 'drs://dg.4DFC:10038031-4300-4c86-b51d-fa227585680f',
   'byte_size': '1078701213',
   'checksum': '0619207c53b3675754ccec92b4ea61ec',
   'data_modality': 'Genomic',
   'imag

In [146]:
len(r[0]['ResearchSubject'])

1

In [151]:
# Make distribution of Research Subject calls
# List specimen per research subjects

research_subject_counts = {}
for research_subject in r:
    research_subject_id = research_subject['id']
    research_subjects_counts = len(research_subject['ResearchSubject'])
    research_subject_counts[research_subject_id] = research_subjects_counts
pd.Series(research_subject_counts).value_counts()

# Why did it not return all? Need to set the limit to be much higher

1    470
dtype: int64

In [179]:
source_material_types = {}
specimen_types = {}
for research_subject in r:
#for research_subject in r[0:1]:
    research_subject_id = research_subject['id']
    specimens = research_subject['ResearchSubject'][0]['Specimen']
    for specimen in specimens:
        specimen_id = specimen['id']
        source_material_type = specimen['source_material_type']
        specimen_type = specimen['specimen_type']
        
        source_material_types[specimen_id] = source_material_type
        specimen_types[specimen_id] = specimen_type

In [180]:
pd.Series(source_material_types).value_counts()

Metastatic               5676
Blood Derived Normal     3583
Primary Tumor            2513
Solid Tissue Normal        34
Additional Metastatic      18
dtype: int64

In [181]:
pd.Series(specimen_types).value_counts()

aliquot    4961
analyte    2360
portion    2157
sample     1384
slide       962
dtype: int64

In [184]:
specimen['File']

[{'id': '969cf1c6-52d1-4353-b712-68c1c5e0d051',
  'identifier': [{'system': 'GDC',
    'value': '969cf1c6-52d1-4353-b712-68c1c5e0d051'}],
  'label': 'TCGA-GN-A262-01Z-00-DX1.64FB584F-24C0-4311-A75F-365CD48D0CD6.svs',
  'data_category': 'Biospecimen',
  'data_type': 'Slide Image',
  'file_format': 'SVS',
  'associated_project': 'TCGA-SKCM',
  'drs_uri': 'drs://dg.4DFC:969cf1c6-52d1-4353-b712-68c1c5e0d051',
  'byte_size': '1458397197',
  'checksum': '1f640abf4eb96c65977478e899001515',
  'data_modality': 'Genomic',
  'imaging_modality': None,
  'dbgap_accession_number': None}]

In [286]:
records = []
for research_subject in r:
#for research_subject in r[0:1]:
    research_subject_id = research_subject['id']
    specimens = research_subject['ResearchSubject'][0]['Specimen']
    for specimen in specimens:
        specimen_id = specimen['id']
        source_material_type = specimen['source_material_type']
        specimen_type = specimen['specimen_type']
        if specimen_type != 'aliquot':
            continue
        
        files = specimen['File']
        for file in files:
            file_format = file['file_format']
            if file_format != 'BAM':
                continue
            else:
                file_id = file['id']
                label = file['label']
                data_category = file['data_category']
                data_modality = file['data_modality']
                data_type = file['data_type']
                byte_size = file['byte_size']
                checksum = file['checksum']
                gdc_identifier = file['identifier'][0]['value']
                
                record = {
                    'research_subject_id': research_subject_id,
                    'specimen_id': specimen_id,
                    'specimen_source_material_type': source_material_type,
                    'specimen_type': specimen_type,
                    'file_id': file_id,
                    'label': label,
                    'data_category': data_category,
                    'data_modality': data_modality,
                    'data_type': data_type,
                    'checksum': checksum,
                    'byte_size': byte_size,
                    'gdc_identifier': gdc_identifier
                }
                records.append(record)

In [287]:
df = pd.DataFrame(records)
len(records)
# This has returned 8928 bams, but on the GDC portal there are only 1,881 bams
# It looks like all of the GDC results are aliquots instead of samples, 


1881

In [288]:
df.columns

Index(['research_subject_id', 'specimen_id', 'specimen_source_material_type',
       'specimen_type', 'file_id', 'label', 'data_category', 'data_modality',
       'data_type', 'checksum', 'byte_size', 'gdc_identifier'],
      dtype='object')

In [289]:
df['specimen_type'].value_counts()

aliquot    1881
Name: specimen_type, dtype: int64

In [290]:
df['checksum'].value_counts().value_counts()

1    1881
Name: checksum, dtype: int64

In [291]:
1404+477
# They are all identical anyway, so we'll filter for aliqots to be consistent with GDC

1881

In [292]:
df['specimen_source_material_type'].value_counts()

Metastatic               1096
Blood Derived Normal      469
Primary Tumor             308
Solid Tissue Normal         6
Additional Metastatic       2
Name: specimen_source_material_type, dtype: int64

In [293]:
# The file type breakdown is
# WXS: 944
# RNA-seq: 472
# miRNA-Seq: 452
# ATAC-seq: 13
# Can we decode what sequencing type each row is based on the labels?

df['label'].str.contains('mirna').value_counts()

False    1429
True      452
Name: label, dtype: int64

In [294]:
# It looks like atac is also included in the filename
# https://portal.gdc.cancer.gov/files/62778ced-3132-4964-a320-824af1f266a2 
df['label'].str.contains('atac').value_counts()

False    1868
True       13
Name: label, dtype: int64

In [295]:
# Four periods means WXS?
df['label'].str.split('.').apply(lambda x: len(x)).value_counts() 

4    944
2    937
Name: label, dtype: int64

In [296]:
df['sequencing_type'] = pd.NA

idx_mi_rna_seq = df.loc[:, 'label'].str.contains('mirna')
df.loc[idx_mi_rna_seq, 'sequencing_type'] = 'miRNA-Seq'

idx_atac_seq = df.loc[:, 'label'].str.contains('atac')
df.loc[idx_atac_seq, 'sequencing_type'] = 'ATAC-Seq'

idx_wes = df['label'].str.split('.').apply(lambda x: len(x)).eq(4)
df.loc[idx_wes, 'sequencing_type'] = 'WES'

df['sequencing_type'].fillna('RNA-seq', inplace=True)

# I suspect that this fix is cohort specific

In [297]:
df['sequencing_type'].value_counts()

WES          944
RNA-seq      472
miRNA-Seq    452
ATAC-Seq      13
Name: sequencing_type, dtype: int64

## Create gdc download manifest
fields:
- id
- filename
- md5
- size
- state

In [298]:
downloaded_manifest = pd.read_csv('gdc-data-portal-files/gdc_manifest_20220328_025435.txt', sep='\t')
generated_manifest = pd.DataFrame(columns=downloaded_manifest.columns)

In [299]:
df.columns

Index(['research_subject_id', 'specimen_id', 'specimen_source_material_type',
       'specimen_type', 'file_id', 'label', 'data_category', 'data_modality',
       'data_type', 'checksum', 'byte_size', 'gdc_identifier',
       'sequencing_type'],
      dtype='object')

In [302]:
downloaded_manifest['state'].value_counts()

validated    1881
Name: state, dtype: int64

In [300]:
df['label'].isin(downloaded_manifest['filename']).value_counts()

True    1881
Name: label, dtype: int64

In [336]:
generated_manifest['id'] = df['file_id']
generated_manifest['filename'] = df['label']
generated_manifest['md5'] = df['checksum']
generated_manifest['size'] = df['byte_size'].astype(int)
generated_manifest['state'] = 'validated' # i'm not sure if this is standard or not

In [337]:
generated_manifest = generated_manifest.sort_values(['id', 'filename']).reset_index(drop=True)
downloaded_manifest = downloaded_manifest.sort_values(['id', 'filename']).reset_index(drop=True)

In [338]:
generated_hashes = pd.util.hash_pandas_object(generated_manifest)
downloaded_hashes = pd.util.hash_pandas_object(downloaded_manifest)

In [339]:
generated_hashes.isin(downloaded_hashes).value_counts()

True    1881
dtype: int64

In [340]:
downloaded_manifest.dtypes

id          object
filename    object
md5         object
size         int64
state       object
dtype: object

In [341]:
generated_manifest.dtypes

id          object
filename    object
md5         object
size         int64
state       object
dtype: object